In [ ]:
# import stuff

import json

In [36]:
#functions to be used
def read_labeled_data_reformat(input_file, output_file):
    # --- PROCESSING ---
    with open(input_file, "r", encoding="utf-8") as f:
        data = json.load(f)

    flattened = []
    for entry in data:
        flat_entry = {} 

        # Add all key-value pairs from the "data" dict
        flat_entry.update(entry.get("data", {}))

        # Extract the first "choices" value from annotations → result
        choices = None
        for ann in entry.get("annotations", []):
            for res in ann.get("result", []):
                val = res.get("value", {})
                if "choices" in val:
                    # if multiple choices exist, join them
                    choices = ", ".join(val["choices"])
                    break
            if choices:
                break

        flat_entry["choices"] = choices

        flattened.append(flat_entry)

    # --- SAVE OUTPUT ---
    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(flattened, f, indent=4, ensure_ascii=False)


import json

def get_choices(json_data, paragraph, sentence_nr):
    """
    Returns the 'choices' value for a given paragraph and sentence number
    from the flattened JSON file.

    Parameters:
        json_file (str): Path to the flattened JSON file
        paragraph (str or int): Paragraph number to look up
        sentence_nr (str or int): Sentence number to look up

    Returns:
        str or None: The choices value if found, otherwise None
    """

    for entry in json_data:
        if str(entry.get("paragraph")) == str(paragraph) and str(entry.get("sentence_nr")) == str(sentence_nr):
            return entry.get("choices")

    return None  # If no match found



In [26]:
input_file_markus = "/work/MarkusLundsfrydJensen#1865/Bachelor_project/json_files/markus_labeling.json"
output_file_markus = "/work/MarkusLundsfrydJensen#1865/Bachelor_project/json_files/cleaned_labelled_markus.json"

read_labeled_data_reformat(input_file_markus, output_file_markus)

In [27]:
input_file_rune = "/work/MarkusLundsfrydJensen#1865/Bachelor_project/json_files/labelled_rune.json"
output_file_rune = "/work/MarkusLundsfrydJensen#1865/Bachelor_project/json_files/cleaned_labelled_rune.json"

read_labeled_data_reformat(input_file_rune, output_file_rune)

In [28]:
#Inter annotater agreement

#load data
with open(output_file_markus, "r", encoding="utf-8") as f:
        markus_data = json.load(f)

with open(output_file_rune, "r", encoding="utf-8") as f:
        rune_data = json.load(f)


In [ ]:
inter_annotater_agreement = []
inter_annotater_disagreement = []

for entry in markus_data:
    paragraph_nr = entry['paragraph']
    sentence_nr = entry['sentence_nr']

    model_blame = entry["Blame"]

    markus_choice = entry["choices"]
    
    rune_choice = get_choices(rune_data, paragraph_nr, sentence_nr)

    if markus_choice != rune_choice:
        inter_annotater_disagreement.append((int(paragraph_nr), int(sentence_nr)))

    elif markus_choice == rune_choice:
        inter_annotater_agreement.append((int(paragraph_nr), int(sentence_nr)))
    
    else:
        print("Error")
    
    

In [47]:
print(len(inter_annotater_agreement))
print(len(inter_annotater_disagreement))

86
14


In [48]:
agreement_with_model = []
disagreement_with_model = []

for entry in markus_data:
    paragraph_nr = entry['paragraph']
    sentence_nr = entry['sentence_nr']

    model_blame = entry["Blame"]

    markus_choice = entry["choices"]
    
    rune_choice = get_choices(rune_data, paragraph_nr, sentence_nr)

    annotater_choice = 0

    if (markus_choice == "Blame") or (rune_choice == "Blame"):
        annotater_choice = 1

    if annotater_choice == int(model_blame):
        agreement_with_model.append((int(paragraph_nr), int(sentence_nr)))
    
    elif annotater_choice != int(model_blame):
        disagreement_with_model.append((int(paragraph_nr), int(sentence_nr)))


In [49]:
print(len(agreement_with_model))
print(len(disagreement_with_model))

78
22
